In [1]:
import sys
import html
import pandas
import pickle
import json
sys.path.append('..')
from aips import *

In [2]:
outdoors_collection="outdoors"
create_collection(outdoors_collection)

upsert_integer_field(outdoors_collection,"post_type_id")
upsert_integer_field(outdoors_collection,"accepted_answer_id")
upsert_integer_field(outdoors_collection,"parent_id")
upsert_integer_field(outdoors_collection,"score")
upsert_integer_field(outdoors_collection,"view_count")
upsert_text_field(outdoors_collection,"body")
upsert_text_field(outdoors_collection,"title")
upsert_keyword_field(outdoors_collection,"tags")
upsert_integer_field(outdoors_collection,"answer_count")

Wiping 'outdoors' collection
[('action', 'CREATE'), ('name', 'outdoors'), ('numShards', 1), ('replicationFactor', 1)]
Creating outdoors' collection
Status: Success
Adding 'post_type_id' field to collection
Status: Success
Adding 'accepted_answer_id' field to collection
Status: Success
Adding 'parent_id' field to collection
Status: Success
Adding 'score' field to collection
Status: Success
Adding 'view_count' field to collection
Status: Success
Adding 'body' field to collection
Status: Success
Adding 'title' field to collection
Status: Success
Adding 'tags' field to collection
Status: Success
Adding 'answer_count' field to collection
Status: Success


In [3]:
from ltr.download import download, extract_tgz
import tarfile

dataset = ['https://github.com/ai-powered-search/outdoors/raw/master/outdoors.tgz']
download(dataset, dest='data/')
extract_tgz('data/outdoors.tgz') # -> Holds 'outdoors.csv', a big CSV file of the stackexchange outdoors dataset

data/outdoors.tgz already exists


In [4]:
def transformData(filename):
    data = []
    df = pandas.read_csv(filename)
    df = df.fillna(0)
    df["id"] = pandas.to_numeric(df["id"], errors='coerce').astype(str)
    df["post_type_id"] = pandas.to_numeric(df["post_type_id"], errors='coerce').astype(int)
    df["accepted_answer_id"] = pandas.to_numeric(df["accepted_answer_id"], errors='coerce').astype(int)
    df["parent_id"] = pandas.to_numeric(df["parent_id"], errors='coerce').astype(int)
    df["score"] = pandas.to_numeric(df["score"], errors='coerce').astype(int)
    df["view_count"] = pandas.to_numeric(df["view_count"], errors='coerce').astype(int)
    df["answer_count"] = pandas.to_numeric(df["answer_count"], errors='coerce').astype(int)
    
    for idx,row in df.iterrows():
        tagset = []
        if row["tags"] and isinstance(row["tags"],str):
            tagset = [t.replace('-',' ') for t in re.compile("[\<\>]").split(html.unescape(row["tags"])) if len(t)]

            data.append({
            "id": row["post_type_id"],
            "post_type_id" : (row["post_type_id"] if row["post_type_id"] else None),
            "accepted_answer_id" : (row["accepted_answer_id"] if row["accepted_answer_id"] else None),
            "parent_id" : (row["parent_id"] if row["parent_id"] else None),
            "score" : (row["score"] if row["score"] else None),
            "view_count" : (row["view_count"] if row["view_count"] else None),
            "body" : row["body"],
            "title" : row["title"],
            "tags" : tagset,
            "answer_count" : (row["answer_count"] if row["answer_count"] else None)
        })
    return data
data = transformData('data/posts.csv')

In [5]:
print("\nAdding Documents to '" + outdoors_collection + "' collection")
response = requests.post(solr_url + outdoors_collection + "/update?commit=true", json=data).json()
print("Status: " "Success" if response["responseHeader"]["status"] == 0 else "Failure" )


Adding Documents to 'outdoors' collection
Status: Success
